# Monitor Linux

2025-06-15

In [1]:
%%writefile message.s
// --------------------------------------------------------------------------
// A simple AArch64 bare-metal-like program for Linux user-space.
// Demonstrates basic stack setup, function calls, and Linux syscalls (write, exit).
// Designed to be loaded and executed by a simple C loader (like load.c).
// --------------------------------------------------------------------------

.global _start
.section .text

// Define ALLOC_SIZE here, it must match load.c
.set ALLOC_SIZE, 512

// --- Linux Syscall Numbers ---
.set SYS_READ,         63
.set SYS_WRITE,        64
.set SYS_EXIT,         93 // Added: Syscall to exit the process

// --- File Descriptors ---
.set STDIN_FD,         0
.set STDOUT_FD,        1

_start:
    // Configure the Stack Pointer (SP)
    // Get the Program Counter (PC) value (address of _start) into x0.
    adr x0, _start
    // Add ALLOC_SIZE to x0 to get the end of our allocated memory block.
    // This address will be used as the initial stack pointer, growing downwards.
    add x0, x0, #ALLOC_SIZE
    // Move the calculated address to the Stack Pointer (SP).
    mov sp, x0

    // --- Output "Ready" message ---
    // Use ADR (Address Register) to get the PC-relative address of ready_msg into x0.
    // ADR is suitable for position-independent code as it calculates the address
    // relative to the current instruction.
    adr x0, ready_msg               // Call puts_aarch64 to print the string
    // Branch with Link (bl) to puts_aarch64.
    // This saves the return address (address of the next instruction) into the Link Register (lr, x30).
    bl puts_aarch64

    // Exit the process after printing the message
    // Load the syscall number for SYS_EXIT (93) into x8.
    mov x8, SYS_EXIT                // Syscall number for exit
    // Set the exit code to 0 (success) in x0.
    mov x0, #0                      // Exit code (0 for success)
    // Execute the syscall. The kernel will terminate the process.
    svc #0                          // Execute the syscall

// --------------------------------------------------------------------------
// Utility Functions (Linux Syscall I/O)
// --------------------------------------------------------------------------

// getchar_aarch64: Reads a character from stdin and returns it in w0 (lowest 32 bits of x0)
// This function uses the stack for its syscall buffer and does NOT explicitly save/restore
// x0/x1 with stp/ldp instructions, as these are handled as caller-saved or not needed by this function.
// (Included for completeness and potential future use, but not called in this version)
getchar_aarch64:
    // Allocate space on the stack for 1 byte for the syscall buffer.
    // We reserve 16 bytes to maintain AArch64 ABI stack alignment requirements.
    sub sp, sp, #16
    
    // Configure arguments for SYS_READ syscall:
    mov x0, STDIN_FD            // x0 = file descriptor (0 for stdin)
    mov x1, sp                  // x1 = buffer address (points to the allocated space on stack)
    mov x2, #1                  // x2 = count (number of bytes to read, 1 in this case)
    mov x8, SYS_READ            // x8 = syscall number (63 for SYS_READ)
    svc #0                      // Execute the syscall (reads 1 byte into [sp, #0])

    // Read the character from the stack buffer into w0.
    ldrb w0, [sp, #0]
    
    // Deallocate the stack space reserved for the buffer.
    add sp, sp, #16
    ret                         // Return to the caller.

// putchar_aarch64: Writes a character from w0 to stdout
// This function uses the stack for its syscall buffer and does NOT explicitly save/restore
// x0/x1 with stp/ldp instructions, as these are handled as caller-saved or not needed by this function.
putchar_aarch64:
    // Allocate space on the stack for 1 byte for the syscall buffer.
    // We reserve 16 bytes to maintain AArch64 ABI stack alignment requirements.
    sub sp, sp, #16
    
    // Store the character from w0 onto the stack, which will serve as the syscall buffer.
    strb w0, [sp, #0]
    
    // Configure arguments for SYS_WRITE syscall:
    mov x0, STDOUT_FD           // x0 = file descriptor (1 for stdout)
    mov x1, sp                  // x1 = buffer address (points to the character on stack)
    mov x2, #1                  // x2 = count (number of bytes to write, 1 in this case)
    mov x8, SYS_WRITE           // x8 = syscall number (64 for SYS_WRITE)
    svc #0                      // Execute the syscall (writes 1 byte from [sp, #0])
    
    // Deallocate the stack space reserved for the buffer.
    add sp, sp, #16
    ret                         // Return to the caller.

// puts_aarch64: Prints a null-terminated string pointed to by x0 to stdout
// This function uses x20 (a callee-saved register) to preserve the string pointer
// across calls to putchar_aarch64, which clobbers x0.
// It also explicitly saves and restores its own lr for robust function calls.
puts_aarch64:
    // Save callee-saved register x20 and the Link Register (lr, x30) onto the stack.
    // This is crucial to preserve them across the function's operations and nested calls.
    // The '!' (pre-index) means sp is decremented BEFORE the store.
    stp x20, lr, [sp, #-16]!    // Store Pair: x20 and lr onto stack, decrement sp by 16.
    
    // Move the initial string pointer (passed in x0) to x20.
    // We use x20 because it's a callee-saved register, meaning its value
    // will be preserved across calls to putchar_aarch64, which modifies x0.
    mov x20, x0                 // Save the string pointer (original x0) into x20
.put_char_loop:
    // Load a byte from the string pointed to by x20 into w1.
    ldrb w1, [x20]              // Load byte from string using x20
    // Compare the loaded character with the null terminator (0).
    cmp w1, #0                  // Compare with null terminator
    // If it's the null terminator, branch to the end of the loop.
    b.eq .put_char_end          // If null, exit loop
    
    // Move the character from w1 to w0 to prepare it as an argument for putchar_aarch64.
    mov w0, w1                  // Move character to w0 (argument for putchar)
    // Branch with Link to putchar_aarch64 to print the character.
    // This updates lr to the instruction immediately following this bl.
    bl putchar_aarch64          // Call putchar_aarch64 (Note: putchar_aarch64 clobbers x0, but our x20 is safe)
    
    // Increment the string pointer (in x20) to point to the next character.
    add x20, x20, #1            // Increment the string pointer (now in x20)
    // Branch unconditionally back to the beginning of the loop.
    b .put_char_loop
.put_char_end:
    // Restore the saved x20 and lr from the stack.
    // The '!' (post-index) means sp is incremented AFTER the load.
    ldp x20, lr, [sp], #16      // Load Pair: Restore x20 and lr, increment sp by 16.
    ret                         // Return to the caller (_start in this case).

// --------------------------------------------------------------------------
// Data within the .text section (placed after all functions)
// --------------------------------------------------------------------------
// Ensure 8-byte alignment for data. This is good practice for AArch64.
.align 8
ready_msg:
    .ascii "Ready\r\n\0" // Null-terminated string for the "Ready" message.
// char_buffer is no longer needed here as putchar_aarch64 and getchar_aarch64
// now use dynamically allocated stack space for their single-byte buffers.

Writing message.s


In [2]:
%%bash
as -o message.o message.s
ld -Ttext=0x0 -nostdlib -o message.elf message.o
objcopy -O binary message.elf message.img

In [4]:
! load message.img

Ready


In [6]:
! size message.elf

   text	   data	    bss	    dec	    hex	filename
    264	      0	      0	    264	    108	message.elf
